# Cifrario di Hill
## Implementiamo il cifrario di Hill in python
Per prima cosa importiamo numpy che gestisce già le moltiplicazioni tra matrici e tra vettori.

In [205]:
import numpy as np
import scipy as sc
from itertools import combinations

Definiamo quindi una funzione che si appoggi alla moltiplicazione matriciale di numpy ma la renda modulo 26.

In [206]:
def mulmod(A,B):
    C=np.dot(A,B)
    d=C.shape
    
    for i in range(d[0]):
        for j in range(d[1]):
            C[i,j]=C[i,j]%26
    return C

Definiamo una lista che colleghi le lettere ai valori numerici.

In [207]:
s="abcdefghijklmnopqrstuvwxyz"
alfabeto=list(s)
r=range(26)
D=dict(zip(alfabeto,r))
DD=dict(zip(r,alfabeto))

Estraiamo il testo da un file txt.

In [208]:
file=open("Plain.txt","r")
PT=[]
for line in file:
    PT.append(line)

file.close()

Ripuliamo il messaggio.

In [209]:
PT="".join(PT)
PT=PT.split(" ")
PT="".join(PT)
PT=PT.split("\n")
PT="".join(PT)
PT=PT.split("'")
PT="".join(PT)
PT=PT.lower()


Trasformiamo il messaggio in un  vettore numerico e lo dividiamo in blocchi di dimensione fissata m con un reshape.

In [210]:
m=3
P=list(PT)
P=[ D[P[i]] for i in range(len(P))]
n=len(P)
print(n)
P=P+[0]*(m-n%m)
#print(P)
P=np.array(P)
print(P)
n=len(P)
righe=int(n/m)


P=np.reshape(P,(m,righe))
P

1477
[13 14 13 ...  4  0  0]


array([[13, 14, 13, ...,  6,  8,  0],
       [ 2,  7,  4, ..., 17, 14,  8],
       [18, 14, 11, ...,  4,  0,  0]])

In [211]:
P[:,295]

array([17, 14,  4])

Definiamo una chiave K

In [212]:
K=np.array([[5,2,4],[15,3,12],[1,7,5]])

Cifriamo tramite la moltiplicazione matriciale modulo 26 e otteniamo il testo cifrato riutilizzando la corridpondenza lettere numeri.

In [213]:
C=mulmod(K,P)
CT=np.reshape(C,(1,n))
CT=CT[0]
CT=[DD[i] for i in CT]
CT="".join(CT)

In [214]:
CT

'lknlcsuiiwxmnxcvmotegistdctkgcnmufuigpcganmtkstuhkgseqbhmgssoowpggwuitetnemtcerqtyqasfwgotunachcgafkavwkdscjstiosakmgesdbfcuomaqauoxxebktoxarulsfyjkoxsbcvognypeuwwdfqhclpyvqwdwgbzyiezomsqaqnyiaseiixqxupgqarqhquqijqcvatxxwemqcymlimzgaibwnujwoidtcjrqatwcckaxwkwshihcxmgolstamtgvfudqvqiweqhlycpvyhhzociyqazurakslvgkkujmfaidovoyhyakwbzmdjywwouxedbfuamrcevqcetkcwpkzydomwagmsamtmciojjqwpaerieprudkoxvivegwetayodihwvelkikogwszzeteidapqbwauhmgcjipwcwsbfjkccbkmwujgegulijsxioasjsuectqbwwmgkaupcdceecqqbjbfgvxylytddbtcuhfflmpxyqysmiluvztmtvbgusysopdwishczjniyisqdaidlncyfpthftlprwmntlyeudhlaznbyuldtacoidoflaalppwajbedstqcwdwisxlgnzamtgducsrnjzvpkvljagqeellcherodobedpvaivkzhqwysuhlamyeaquvklsiqcwgltlbbosqfnbfrsqsbrqntqbefwfoailjruvnowbyxnfgqlapedzmkwqebenmqowpxnfzmkmvvjhaxnusriwezdqpayobwumveaihdlikgtssqavciguueddsrsztkqeovkqgbryujthvoogbdjydwlyawamkwwhxvcgzhotgyfoffdkkndiqrbjwbvyegmnhaslyahzavancktajrjacodnherxatoasyosuazcmktonkcytagmrqgemfcqgtafbmhkquguvjmlrjlusqbpznqdweoqecrqwryqunahgyndsdifttvvbtd

## Inversione e Decifratura
Costruiamo una funzione che calocli, se possibile, la matrice inversa di K e usiamola per decifrare il testo.

In [215]:
def invertibile(A):
    det=round(sc.linalg.det(A))
    return not (det%13==0 or det%2==0)

    

In [216]:
def inversomod(x,num):
    a=x
    b=num
    
    while a%b>0:
        r=a%b
        q=a//b
        a=b
        b=r
    return b
inversomod(112,12)

4

In [217]:
def inversa(A):
    det=round(sc.linalg.det(A)%26)
    print(det)
    invdet=0
    if det%26==2 or det%26==13 or det%26==0:
        return 0
    while det*invdet%26!=1:
        invdet+=1
    
    print(invdet)
    inv=[]
    d=A.shape
    
    for i in range(d[0]):
        riga=[]
        for j in range(d[1]):
            parita=(-1)**(((i+1)+(j+1)))
            rid=ridotta(A,j,i)
            #print(rid.shape)
            if 1 in rid.shape:
                detrid=rid[0]
            else:
                detrid=round(sc.linalg.det(rid)%26)
            #print(type(parita), type(invdet), type(detrid))
            riga.append((parita*detrid*invdet)%26)
        inv.append(riga)
    return np.array(inv)

In [218]:
def ridotta(A,riga,colonna):
    d=A.shape
    rid=[]
    for i in range(d[0]):
        l=[]
        if i!=riga:
            for j in range(d[1]):
                if j!=colonna:
                    l.append(A[i,j])
            rid.append(l)
    return np.array(rid)


In [219]:
Inv=inversa(K)
Inv

15
7


array([[11, 22,  6],
       [ 1, 17,  0],
       [12,  3, 25]])

In [220]:
np.dot(K,Inv)%26

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=int32)

In [221]:
PP=np.dot(Inv,C)%26
PPP=np.reshape(PP,(1,n))[0]
PPP="".join([DD[i] for i in PPP])
PPP

'nonpuoicombattereunaguerradasoloilcuoreeunarmaturacisalvamasiconsumaavoltechiedereaiutocifapauramabastaunsolopassocomeilprimouomosullalunaperchedafuorinonsivedequantevoltehaipiantosinascesoliesimuorenelcuorediqualcunaltrosiamoangeliconunalasoltantoeriusciremoavolaresolorestandolunoaccantoallaltrocammineroaunpassodateefermeremoilventocomedentroagliuraganisupereroicomeioeteseavraipauraallorastringimilemaniperchesiamoinvincibilivicinieovunqueandrosaraiconmesupereroisoloioeteduegoccedipioggiachesalvanoilmondodallenuvolecisonoferitechenonsenevannonemmenocoltempopiuprofondediquellochesembranoguarisconosopralapellemainfondoticambianodentrohoversatocositantelacrimefinoadodiaremestessomaognivoltachehotoccatoilfondotucerilostessoohquandosiamodistantiognivoltachepiangipiangepureilcieloohnonhomoltodadartimatigiurochecammineroaunpassodateefermeremoilventocomedentroagliuraganisupereroicomeioeteseavraipauraallorastringimilemaniperchesiamoinvincibilivicinieovunqueandrosaraiconmesupereroisoloioetedueg

## Attacco

In [222]:

lista_indici=list(range(len(P[0])))
indici=combinations(lista_indici,m)
indice=next(indici)
while not invertibile(P[:,indice]):
    
    #print(indice,invertibile(P[:,indice]))
    indice=next(indici)
print(indice)

(0, 1, 2)


In [223]:
round(sc.linalg.det(P[:,indice])%26)

11

In [224]:
Past=P[:,indice]
Pastm1=inversa(Past)
Kstar=np.dot(C[:,indice],Pastm1)%26
Kstar

11
19


array([[ 5,  2,  4],
       [15,  3, 12],
       [ 1,  7,  5]], dtype=int32)

In [225]:
Kstar==K

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])